In [1]:
!pip install -q kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

## download dataset 
!kaggle datasets download -d mirichoi0218/insurance

## unzip dataset
!unzip "insurance.zip" -d "insurance_dataset"

     |████████████████████████████████| 61kB 3.5MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=766105b1e273f9c8652a4201c5526588589c9dfbb1e693762533b4442e79f1d5
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Saving kaggle.json to kaggle.json
kaggle.json
  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 13.6MB/s]
Archive:  insurance.zip
  inflating: insurance_dataset/insurance.csv  


## Import libraries

In [2]:
import pandas as pd


In [3]:
data = pd.read_csv('/content/insurance_dataset/insurance.csv')

In [4]:
data.shape

(1338, 7)

In [5]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [11]:
len(data.loc[data.duplicated()])

0

In [10]:
data = data.drop_duplicates()

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1337 non-null   int64  
 1   sex       1337 non-null   int64  
 2   bmi       1337 non-null   float64
 3   children  1337 non-null   int64  
 4   smoker    1337 non-null   int64  
 5   region    1337 non-null   int64  
 6   charges   1337 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 83.6 KB


In [14]:
def col_unique_values(col_name):
  ## input : category variables
  ## Output : number of unique values in particular category col
  print(f"****************** Col Name : {col_name} ****************")
  print(f"Unique Values :- \n {data[col_name].unique()}")
  print(f"Number of Unique values :- {data[col_name].nunique()}\n\n")


## all columns
total_col_names = data.columns
## find numeric columns (int & float, bool)
num_cols = data._get_numeric_data().columns
## getting category columns 
cat_col_names = list(set(total_col_names) - set(num_cols))


for col_name in cat_col_names:
  ## check unique values of every category column
  col_unique_values(col_name)

****************** Col Name : sex ****************
Unique Values :- 
 ['female' 'male']
Number of Unique values :- 2


****************** Col Name : smoker ****************
Unique Values :- 
 ['yes' 'no']
Number of Unique values :- 2


****************** Col Name : region ****************
Unique Values :- 
 ['southwest' 'southeast' 'northwest' 'northeast']
Number of Unique values :- 4




In [12]:
from sklearn.preprocessing import LabelEncoder

In [16]:
## Apply label encoding operation on category columns
def label_encoding(col_name):
  le = LabelEncoder()
  data[col_name] = le.fit_transform(data[col_name])


for col_name in cat_col_names:
  label_encoding(col_name)

In [18]:
from sklearn.model_selection import train_test_split

In [53]:
X = data.drop(['charges'], axis=1)
y = data['charges']

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures

In [54]:
column_names = list(X.columns)

## create pipeline to apply feature scaling
pipeline = Pipeline([
                     ('poly_features', PolynomialFeatures(degree=2))
])

## apply feature scaling on independent values (X)
X = pipeline.fit_transform(X)
# X = pd.DataFrame(data=pipeline.fit_transform(X), columns=column_names)

In [115]:
X1 = data.drop(['charges'], axis=1)
y1 = data['charges']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.1, random_state = 42)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

In [56]:
from sklearn.linear_model import LinearRegression

In [57]:
lr_re = LinearRegression()
lr_re.fit(X_train, y_train)
lr_re.score(X_test, y_test)

0.8955446604737745

In [58]:
from sklearn.ensemble import RandomForestRegressor

In [62]:
rf_re = RandomForestRegressor()
rf_re.fit(X_train, y_train)
rf_re.score(X_test, y_test)

0.8708340158047627

In [61]:
rf_re = RandomForestRegressor(criterion='mae')
rf_re.fit(X_train, y_train)
rf_re.score(X_test, y_test)

0.8584214990266331

In [63]:
from sklearn.tree import DecisionTreeRegressor

In [67]:
dt_re = DecisionTreeRegressor()
dt_re.fit(X_train, y_train)
dt_re.score(X_test, y_test)

0.7662696582335776

In [82]:
from sklearn.linear_model import Ridge, Lasso

In [119]:
r_re = Ridge(alpha=1.0)
r_re.fit(X_train, y_train)
r_re.score(X_test, y_test)

0.8945532910149899

In [81]:
r_re = Ridge(alpha=0.1)
r_re.fit(X_train, y_train)
r_re.score(X_test, y_test)

0.8952602573996508

In [84]:
l_re = Lasso(alpha=0.1)
l_re.fit(X_train, y_train)
l_re.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13681583620.580353, tolerance: 17185940.079363473
  positive)


0.8953510783604246

In [86]:
from sklearn.ensemble import AdaBoostRegressor

In [117]:
ada_re = AdaBoostRegressor(base_estimator=LinearRegression(), random_state=42, n_estimators=100, learning_rate=1)
ada_re.fit(X_train, y_train)
ada_re.score(X_test, y_test)

0.7742299768894101

In [100]:
from sklearn.ensemble import GradientBoostingRegressor

In [116]:
model = GradientBoostingRegressor(learning_rate=0.1, random_state=42, criterion='mae', )
model.fit(X_train1, y_train1)
model.score(X_test1, y_test1)

0.8930350535982734

In [120]:
from sklearn.linear_model import BayesianRidge

In [124]:
model = BayesianRidge(alpha_init=True, lambda_init=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8899290540556793

In [125]:
from sklearn.linear_model import ElasticNet

In [126]:
model = ElasticNet(alpha=1.0, random_state=1)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8729784073541851

In [138]:
model = ElasticNet(alpha=0.01, random_state=42, precompute=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7814167485682283

In [139]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [140]:
y_predict = lr_re.predict(X_test)

In [141]:
r2_score(y_test, y_predict)

0.8955446604737746

In [142]:
mean_absolute_error(y_test, y_predict)

2699.1012238246267

In [143]:
mean_squared_error(y_test, y_predict)

18748597.35987959